# 라이브러리 설정

In [125]:
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score

In [126]:
pm = pd.read_csv('data/pm_cls.csv') #전처리가 완료된 csv파일 불러오기
pm.reset_index(drop=True,inplace=True) #인덱스 초기화
#pm.drop('Unnamed: 0',axis=1,inplace=True) #필요없는 컬럼 제거
pm.head() #상위 5개의 csv 관찰

,제품 코드,제품명,원재료명,함량(%),분류
0,MI-151007-004,칠리소스,토마토페이스트,38.174524,소스
1,MI-151007-004,칠리소스,양파,13.159478,소스
2,MI-151007-004,칠리소스,양송이,11.574724,소스
3,MI-151007-004,칠리소스,향신지미베이스,7.582748,소스
4,MI-151007-004,칠리소스,베이컨,5.917753,소스


In [127]:
! pip3 install --target=$pg_path gensim==3.8.0 #gensim v3.8.0 이상에서 해야 로딩을 할 수 있음


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip


In [128]:
pm.drop('제품 코드',axis=1,inplace=True)

# 정답 데이터셋 컬럼 통합하기

#### 정답 데이터셋에 대한 데이터프레임 만들기

In [129]:
pulmu = pm.copy()
pulmu

,제품명,원재료명,함량(%),분류
0,칠리소스,토마토페이스트,38.174524,소스
1,칠리소스,양파,13.159478,소스
2,칠리소스,양송이,11.574724,소스
3,칠리소스,향신지미베이스,7.582748,소스
4,칠리소스,베이컨,5.917753,소스
...,...,...,...,...
14900,키즈밀_22_실온소스_S4_한돈짜장덮밥소스,멸치가루,0.010000,소스
14901,키즈밀_22_실온소스_S4_한돈짜장덮밥소스,이소말토올리고당,1.200000,소스
14902,키즈밀_22_실온소스_S4_한돈짜장덮밥소스,혼합제제,2.000000,소스
14903,키즈밀_22_실온소스_S4_한돈짜장덮밥소스,우리밀짜장가루,3.300000,소스


#### FastText Training

In [130]:
import gensim
from gensim import models
from gensim.models import fasttext
from gensim.models.fasttext import FastText
from gensim.models import Word2Vec, KeyedVectors

In [131]:
#solution1
X_train = list(pulmu['제품명'].unique())

In [132]:
#solution2
X_train1 = list(pulmu['제품명'].unique())
X_train2 = list(pulmu['원재료명'].unique())
X_train = X_train1 + X_train2

In [133]:
%%time
ft_model = fasttext.load_facebook_model('data/wiki.ko.bin') #FastText 불러오기
ft_model.build_vocab(sentences=X_train, update=True)
ft_model.train(X_train, total_examples=len(X_train),epochs=10)

CPU times: user 1min 26s, sys: 36.5 s, total: 2min 3s
Wall time: 2min 3s


데이터 프레임 조작

In [134]:
total = ['0']*len(pulmu)

for i in range(len(pm)):
    total[i] = pulmu['원재료명'][i] + ' ' + str(round(pulmu['함량(%)'][i],2))

In [135]:
pulmu['계'] = total

In [136]:
pulmu['함량(%)']=1

In [137]:
pulmu.drop('원재료명',axis=1,inplace=True)
pulmu

,제품명,함량(%),분류,계
0,칠리소스,1,소스,토마토페이스트 38.17
1,칠리소스,1,소스,양파 13.16
2,칠리소스,1,소스,양송이 11.57
3,칠리소스,1,소스,향신지미베이스 7.58
4,칠리소스,1,소스,베이컨 5.92
...,...,...,...,...
14900,키즈밀_22_실온소스_S4_한돈짜장덮밥소스,1,소스,멸치가루 0.01
14901,키즈밀_22_실온소스_S4_한돈짜장덮밥소스,1,소스,이소말토올리고당 1.2
14902,키즈밀_22_실온소스_S4_한돈짜장덮밥소스,1,소스,혼합제제 2.0
14903,키즈밀_22_실온소스_S4_한돈짜장덮밥소스,1,소스,우리밀짜장가루 3.3


원재료명 + 함량을 원재료명에 한번에 표현.

In [138]:
pulmu.columns = ['제품명','함량(%)','분류','원재료명']

In [139]:
pulmu = pulmu[['제품명','원재료명','함량(%)','분류']]
pulmu

,제품명,원재료명,함량(%),분류
0,칠리소스,토마토페이스트 38.17,1,소스
1,칠리소스,양파 13.16,1,소스
2,칠리소스,양송이 11.57,1,소스
3,칠리소스,향신지미베이스 7.58,1,소스
4,칠리소스,베이컨 5.92,1,소스
...,...,...,...,...
14900,키즈밀_22_실온소스_S4_한돈짜장덮밥소스,멸치가루 0.01,1,소스
14901,키즈밀_22_실온소스_S4_한돈짜장덮밥소스,이소말토올리고당 1.2,1,소스
14902,키즈밀_22_실온소스_S4_한돈짜장덮밥소스,혼합제제 2.0,1,소스
14903,키즈밀_22_실온소스_S4_한돈짜장덮밥소스,우리밀짜장가루 3.3,1,소스


# FastText + Node2Vec

In [140]:
pip install node2vec #Node2Vec 설치


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [141]:
pip install --upgrade scipy networkx


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [142]:
import networkx as nx
from node2vec import Node2Vec

#### Node2Vec 모델 훈련

첫번째 model은 함량 정보를 담고있는 제품-원재료명 훈련 모델

In [143]:
pulmu = pulmu.copy()
G = nx.Graph()

#product = pulmu['제품명'].unique().tolist()

# # edge
for i in range(len(pulmu)):
    G.add_edge(pulmu['제품명'][i],pulmu['원재료명'][i],relation='ingredients')


# 노드 임베딩
node2vec = Node2Vec(G,dimensions=20,walk_length=16,num_walks=100)
model = node2vec.fit(window=10,min_count=1)

Computing transition probabilities: 100%|█| 7894/7894 [00:00<0
Generating walks (CPU: 1): 100%|█| 100/100 [05:55<00:00,  3.55


두번째 model1은 함량 정보를 담지않은 제품-원재료명 훈련 모델

In [144]:
G1 = nx.Graph()

#product = pulmu['제품명'].unique().tolist()

# # edge
for i in range(len(pm)):
    G1.add_edge(pm['제품명'][i],pm['원재료명'][i],relation='ingredients')
#ingredient = pulmu['원재료명'].unique().tolist()
#product = pulmu['제품명'].unique().tolist()

# 노드 임베딩
node2vec = Node2Vec(G1,dimensions=20,walk_length=16,num_walks=100)
model1 = node2vec.fit(window=10,min_count=1)

Computing transition probabilities: 100%|█| 2343/2343 [00:03<0
Generating walks (CPU: 1): 100%|█| 100/100 [02:08<00:00,  1.29


# Product's ingredient node

첫번째 섹션

입력값과 유사한 단어(제품) top3를 찾고, 관련 ingredient node를 뽑는 작업을 합니다.

#### 입력 제품명과 가장 유사한 단어

In [151]:
import sys
# Product
def rank_list(name1,name2):
    rank = {}
    product_list = []
    clf = pulmu[pulmu['분류']==name2]
    X_train = list(clf['제품명'].unique())
    for i in range(len(X_train)):
        sim = ft_model.wv.similarity(name1,X_train[i])
        rank[X_train[i]]=sim
    ranked_list = sorted(rank.items(), key=lambda x: x[1],reverse=True)
    ranked_list = ranked_list[:5]
    ranked_list
    for i in range(5):
        product_list.append(ranked_list[i][0])
    return product_list

# ingredient
def node2ingredient(input_data):
    data= model.wv.most_similar(input_data,topn=100)

    answer_ingredient_list =[]

    for i in range(len(data)):
        for j in range(len(ingredient)):
            if(data[i][0]==ingredient[j]):
                answer_ingredient_list.append(data[i])
    ingredient_node =[]
    ingredient_node_sim =[]

    for i in range(len(answer_ingredient_list)):
        ingredient_node.append(answer_ingredient_list[i][0])
        ingredient_node_sim.append(answer_ingredient_list[i][1])

    ingredient_node_df = pd.DataFrame(data=list(zip(ingredient_node,ingredient_node_sim)),columns=['원재료','유사도'])
    ind = ingredient_node_df[:20]
    return list(ind['원재료'])

# 함량 백분율 통합
def integrate(ingred_list):
    dic = {}
    for i in ingred_list:    
        idx = i.index(' ')
        dic[i[0:idx]]=float(i[idx:])
    return dic

In [525]:
ingredient = pulmu['원재료명'].unique().tolist()
product = pulmu['제품명'].unique().tolist()

print('입력 형태 : 제품명,분류명 [ 예시 : 흑임자,소스  ]')
input_data1,input_data2 = map(str,input('제품을 넣어주세요 :').split(',')) #이름과 분류를 입력.
similar_rank = rank_list(input_data1,input_data2) #입력한 이름과 비슷한 product를 추출
top1_similar,top2_similar,top3_similar = similar_rank[0],similar_rank[1],similar_rank[2]

##################################################
print('★ Top_1 similar product is %s' %top1_similar)
print('★ Top_2 similar product is %s' %top2_similar)
print('★ Top_3 similar product is %s' %top3_similar)
##################################################

ingred_list,ingred_list_2,ingred_list_3 = node2ingredient(top1_similar),node2ingredient(top2_similar),node2ingredient(top3_similar)

# print('***********************************************************' * 2)
# print("★ Top_1 similar product's ingredient is %s" %ingred_list)
# print('***********************************************************' * 2)

# print("★ Top_2 similar product's ingredient is %s" %ingred_list_2)
# print('***********************************************************' * 2)

# print("★ Top_3 similar product's ingredient is %s" %ingred_list_3)
# print('***********************************************************' * 2)

dic1,dic2,dic3 = integrate(ingred_list), integrate(ingred_list_2), integrate(ingred_list_3)

dic1 = sorted(dic1.items(), key = lambda x:x[1],reverse=True)
dic2 = sorted(dic2.items(), key = lambda x:x[1],reverse=True)
dic3 = sorted(dic3.items(), key = lambda x:x[1],reverse=True)

# for i in dic1:
#     print(i)
# print('***********************************************************' * 2)
# for i in dic2:
#     print(i)
# print('***********************************************************' * 2)
# for i in dic3:
#     print(i)

입력 형태 : 제품명,분류명 [ 예시 : 흑임자,소스  ]


제품을 넣어주세요 : 부대찌개,양념


★ Top_1 similar product is 바지락된장찌개양념
★ Top_2 similar product is 차돌박이 된장찌개
★ Top_3 similar product is 정통부대찌개찌개양념(19년도)


In [526]:
df1,df2,df3 = pd.DataFrame(dic1),pd.DataFrame(dic2),pd.DataFrame(dic3)
df4 = pd.concat([df1,df2,df3],axis=1)
df4.fillna(' ',inplace=True)
df4.columns = [['top1','top1','top2','top2','top3','top3'], ['원재료','함량','원재료','함량','원재료','함량']]

In [527]:
df4 #top1,top2,top3 제품이 가지고 있는 관계 재료 노드 (가시성을 위해서 밑의 결과로 표현)

top1                    top2                    top3       
             원재료     함량              원재료     함량              원재료     함량
0            정제수  46.47              정제수  51.10            볶음양념분  14.81
1          오월의된장  21.00               두부  15.05       사골엑기스12-16   6.50
2        개량메주발효물  14.00             재래된장   8.02        사골추출농축액-1   6.50
3             마늘   4.44              애호박   7.52       멸치다시마베이스1호   5.20
4        조미분말베이스   3.30               양파   5.02            매운맛분말   4.68
5        꽃게풍미베이스   3.00             차돌양지   4.01              정제염   3.90
6       게추출농축액1호   2.50               고추   2.01        사골풍미인핸서1호   3.90
7        바지락엑기스P   1.60             찹쌀가루   1.00             고추가루   2.60
8      새우추출농축액1호   1.50               대파   1.00               양파   2.60
9         생홍합엑기스   1.48     고추장(프리미엄고추장)   1.00            오월의된장   2.60
10      콩나물추출농축액   1.11             차돌박이   1.00           감칠맛베이스   2.60
11           고추분   1.11          멸치추출농축액   0.50        우마미복합베이스1   2.60
12        감칠맛베이스   0.80               마늘   0.50         쇠고기추출분말P   2.60
13      다시마엑기스-1   0.74         다시마엑기스-1   0.40          멸치추출농축액   2.60
14         발효맛내기   0.70         사골육수조미분p   0.20          아미노베이스P   1.30
15       호박산이나트륨   0.60         소고기밑국물분말   0.20  5'-리보뉴클레오티드이나트륨   1.30
16         꽃게엑기스   0.50             고춧가루   0.20            발효농축액   1.30
17           백설탕   0.30           감칠맛발란스   0.10           청양고춧가루   0.52
18  소고기엑기스페이스트-P   0.20  5'-리보뉴클레오티드이나트륨   0.10            흑후추분말   0.26
19          멸치분말   0.15            아로마일드   0.05         파프리카추출색소   0.13

# 입력값과 유사한 원재료명들의 Node2Vec 관찰

두번째 섹션에서는 입력값과 유사한 원재료명을 찾고(FastText),

위의 결과에서 나온 원재료명의 Node2Vec찾습니다 (Node2Vec).

dic1,dic2,dic3는 입력 제품명과 유사도가 비슷한 제품이 가지고 있는 노드 ingredient 입니다.

In [528]:
new_dict = dic1 + dic2 + dic3

#### (1) 위의 유사한 제품명들이 갖고있는 ingredient node를 데이터프레임화

In [529]:
combination_by_product_df = pd.DataFrame(new_dict)
combination_by_product_df #df4와 같은 결과지만, 다른 표현

,0,1
0,정제수,46.47
1,오월의된장,21.00
2,개량메주발효물,14.00
3,마늘,4.44
4,조미분말베이스,3.30
5,꽃게풍미베이스,3.00
6,게추출농축액1호,2.50
7,바지락엑기스P,1.60
8,새우추출농축액1호,1.50
9,생홍합엑기스,1.48


입력값과 유사도가 비슷한 원재료명 rank

In [530]:
rank = {}
ingredient_list = []
X_train = list(pm['원재료명'].unique())
for i in range(len(X_train)):
    sim = ft_model.wv.similarity(input_data1,X_train[i])
    rank[X_train[i]]=sim
ranked_list = sorted(rank.items(), key=lambda x: x[1],reverse=True)
ranked_list = ranked_list[:10]
ranked_list
for i in range(5):
    ingredient_list.append(ranked_list[i][0])
print(ranked_list)

[('육개장볶음밥소스', 0.66859794), ('된장찌개베이스', 0.6552696), ('찌개베이스1', 0.6531563), ('육개장볶음다대기', 0.65114343), ('육개장볶음다대기-R', 0.64839137), ('육개장볶음다대기분말', 0.6464048), ('찌개양념베이스', 0.6393306), ('육개장볶음다대기분말-R', 0.62411475), ('한식메주된장', 0.6225409), ('비빔밥고추장소스', 0.6221875)]


입력과 유사한 원재료명들의 key값 리스트

In [531]:
new_ranked_list = list(itertools.chain.from_iterable(ranked_list))
new_ranked_list = new_ranked_list[0::2]
new_ranked_list

['육개장볶음밥소스',
 '된장찌개베이스',
 '찌개베이스1',
 '육개장볶음다대기',
 '육개장볶음다대기-R',
 '육개장볶음다대기분말',
 '찌개양념베이스',
 '육개장볶음다대기분말-R',
 '한식메주된장',
 '비빔밥고추장소스']

위의 각 key에 대한 ingredient노드를 추출

##### ★ ingredient Rank3 (입력값과 유사도가 높은 원재료명 top3)

In [532]:
new_ranked_list[0:3]

['육개장볶음밥소스', '된장찌개베이스', '찌개베이스1']

In [533]:
ingredient = pm['원재료명'].unique().tolist()
product = pulmu['제품명'].unique().tolist()

def node2ingredient_no_ingred(input_data):
    data= model1.wv.most_similar(input_data,topn=100)

    answer_ingredient_list =[]

    for i in range(len(data)):
        for j in range(len(ingredient)):
            if(data[i][0]==ingredient[j]):
                answer_ingredient_list.append(data[i])
    ingredient_node =[]
    ingredient_node_sim =[]

    for i in range(len(answer_ingredient_list)):
        ingredient_node.append(answer_ingredient_list[i][0])
        ingredient_node_sim.append(answer_ingredient_list[i][1])

    ingredient_node_df = pd.DataFrame(data=list(zip(ingredient_node,ingredient_node_sim)),columns=['원재료','유사도'])
    ind = ingredient_node_df[:20]
    return list(ind['원재료'])

##### ★ ingredient Rank3's node ingredient (위의 top3 각각의 인접하는 재료 노드)

In [534]:
list_ = list(set(node2ingredient_no_ingred(new_ranked_list[0]) + node2ingredient_no_ingred(new_ranked_list[1]) + node2ingredient_no_ingred(new_ranked_list[2])))
print(list_)

['혼합양념1', '숙성김치', '국물양념2', '오월의된장', '조미분말', '고추분', '지미베이스분말A', '다대기페이스트', '멸치감칠맛분말', '개량메주발효물', '바른선솜씨좋은시골된장', '알파찹쌀분말', '야채탕양념', '대파', '국산콩콩비지', '짬뽕용돼지고기', '가루찹살', '국물양념1', '우렁엑기스1호', '해물베이스', '조미분말베이스', '엠에스씨-잔탄검', '숯불갈비양념', '해마쇠고기다시', '참치조미액', '새우젓분말', '고추', '소고기', '느타리버섯', '우렁', '생홍합엑기스', '사골풍미인헨서', '사골추출농축액-1', '풍미인핸서', '탈염농축간장', '우거지', '국산채소밑국물', '건미역', '꽃게풍미베이스', '삶은우양지', '묵은지엑기스', '육개장조미분말', '육개장볶음다대기', '콩발효맛내기진', '숙주', '매운양념분말', '콩가루', '돼지고기베이스', '볶음김치풍미유', '양지추출액', '두부', '쇠고기', '쇠고기맛발란스', '차돌박이', '숙주나물', '고사리', '진한맛베이스', '카카오색소', '바지락농축액P']


#### (2) 추출된 노드들을 데이터프레임화

In [535]:
combination_by_ingredient_df=pd.DataFrame(list_)

#### (1)과 (2)를 병합

##### ★merge 결과

In [536]:
merge_df = pd.merge(combination_by_product_df,combination_by_ingredient_df)
merge_df

,0,1
0,오월의된장,21.00
1,오월의된장,2.60
2,개량메주발효물,14.00
3,조미분말베이스,3.30
4,꽃게풍미베이스,3.00
5,생홍합엑기스,1.48
6,고추분,1.11
7,두부,15.05
8,고추,2.01
9,대파,1.00


데이터프레임1의 TOP3 제품 각각의 상단에서 함량순3, 중단에서 함량순3, 하단에서 함량순3을 뽑기 위한 작업

### 섹션1의 결과를 가지고 combination을 합니다.

In [537]:
import itertools

_dic1 = list(itertools.chain.from_iterable(dic1))
_dic1 = _dic1[0::2]
_dic2 = list(itertools.chain.from_iterable(dic2))
_dic2 = _dic2[0::2]
_dic3 = list(itertools.chain.from_iterable(dic3))
_dic3 = _dic3[0::2]

In [538]:
result = ()
result = set(_dic1[0:int(len(_dic1)*0.33)])
result

{'개량메주발효물', '꽃게풍미베이스', '마늘', '오월의된장', '정제수', '조미분말베이스'}

In [539]:
cnt = 0
start = int(len(_dic2)*0.33)
end = int(len(_dic2)*0.66)

for v2 in _dic2[start:end] :
    
    result.add(v2)
    cnt = len(result)
    
    if cnt == 6 :
        break
    
result

{'개량메주발효물',
 '고추',
 '고추장(프리미엄고추장)',
 '꽃게풍미베이스',
 '대파',
 '마늘',
 '멸치추출농축액',
 '오월의된장',
 '정제수',
 '조미분말베이스',
 '차돌박이',
 '찹쌀가루'}

In [540]:
cnt = 0
start = int(len(_dic2)*0.66)


for v3 in _dic3[start:] :
    result.add(v3)
    cnt = len(result)
    
    if cnt == 9 :
        break
    
combination = result

##### ▲top3 각각의 상,중,하 ingredient 추출 작업 끝

---------------------------------------------------------------------------------------------------

위의 결과에 해당하는 값들만 뽑음

##### ★ Combination 결과

In [541]:
combination

{"5'-리보뉴클레오티드이나트륨",
 '개량메주발효물',
 '고추',
 '고추장(프리미엄고추장)',
 '꽃게풍미베이스',
 '대파',
 '마늘',
 '멸치추출농축액',
 '발효농축액',
 '아미노베이스P',
 '오월의된장',
 '정제수',
 '조미분말베이스',
 '차돌박이',
 '찹쌀가루',
 '청양고춧가루',
 '파프리카추출색소',
 '흑후추분말'}

In [542]:
df10 = pd.DataFrame()

for i in combination:
    df10 = df10.append(combination_by_product_df[combination_by_product_df[0]==i])
#df10

#MERGE데이터프레임과 조합데이터프레임을 concat하고, 중복된 값은 하나만 남김(함량은 평균내고) ->final데이터프레임

In [543]:
final_df = pd.concat([merge_df, df10])

In [544]:
result = final_df.groupby(0)[1].mean()
result = pd.DataFrame(result)
result.reset_index(drop=False, inplace= True)
result.columns = ['원재료명','함량']

In [545]:
result

,원재료명,함량
0,5'-리보뉴클레오티드이나트륨,0.700
1,개량메주발효물,14.000
2,고추,2.010
3,고추분,1.110
4,고추장(프리미엄고추장),1.000
5,꽃게풍미베이스,3.000
6,대파,1.000
7,두부,15.050
8,마늘,2.470
9,멸치추출농축액,1.550


##### ★백분율 통합

In [546]:
f_df = result.copy()
p = 100 / f_df['함량'].sum(axis=0)
f_df['함량'] = f_df['함량'] * p
f_df

,원재료명,함량
0,5'-리보뉴클레오티드이나트륨,0.586928
1,개량메주발효물,11.738565
2,고추,1.685323
3,고추분,0.930701
4,고추장(프리미엄고추장),0.838469
5,꽃게풍미베이스,2.515407
6,대파,0.838469
7,두부,12.618958
8,마늘,2.071018
9,멸치추출농축액,1.299627


함량 재조정 (ROOT)

In [598]:
import numpy as np

f_df = result.copy()
f_df['함량'] = np.sqrt(f_df['함량'])

p = 100 / f_df['함량'].sum(axis=0)
f_df['함량'] = f_df['함량'] * p
f_df

KeyError: '함량'

##### 강된장,순두부찌개,부대찌개

In [464]:
node2_ingredient1 = list(f_df1['원재료명'])

In [571]:
node2_ingredient1

['가쓰오부시엑기스1호',
 '감자전분',
 '개량메주발효물',
 '건표고버섯',
 '고춧가루',
 '국산채소밑국물',
 '냉동대파',
 '냉동양파',
 '다시마엑기스-1',
 '대파',
 '돼지고기',
 '멸치분말',
 '샘표된장',
 '생강',
 '세이버리메이트1호',
 '식용유',
 '양파',
 '오월의된장',
 '옥수수기름',
 '올리고당',
 '정제수',
 '조미분말',
 '참맛고추장골드',
 '탈염농축간장',
 '포크엑기스P']

In [520]:
node2_ingredient2 = list(f_df2['원재료명'])

In [548]:
node2_ingredient3 = list(f_df3['원재료명'])

In [552]:
pm_df = pm.copy()
pm_df['함량(%)'] = 1
pm_df = pm_df.drop('분류',axis=1)
pm_df

,제품명,원재료명,함량(%)
0,칠리소스,토마토페이스트,1
1,칠리소스,양파,1
2,칠리소스,양송이,1
3,칠리소스,향신지미베이스,1
4,칠리소스,베이컨,1
...,...,...,...
14900,키즈밀_22_실온소스_S4_한돈짜장덮밥소스,멸치가루,1
14901,키즈밀_22_실온소스_S4_한돈짜장덮밥소스,이소말토올리고당,1
14902,키즈밀_22_실온소스_S4_한돈짜장덮밥소스,혼합제제,1
14903,키즈밀_22_실온소스_S4_한돈짜장덮밥소스,우리밀짜장가루,1


In [553]:
l1 = len(node2_ingredient1)
l2 = len(node2_ingredient2)
l3 = len(node2_ingredient3)

d1 = ['강된장_pred_node']*l1 + ['순두부찌개_pred_node'] * l2 + ['부대찌개_pred_node'] * l3
d2 = node2_ingredient1 + node2_ingredient2 + node2_ingredient3
d3 = [1]*(l1+l2+l3)

df = pd.DataFrame([d1,d2,d3])
df = df.transpose()
df.columns = ['제품명', '원재료명', '함량(%)']
result = pd.concat([pm_df,df],ignore_index=True)
result

,제품명,원재료명,함량(%)
0,칠리소스,토마토페이스트,1
1,칠리소스,양파,1
2,칠리소스,양송이,1
3,칠리소스,향신지미베이스,1
4,칠리소스,베이컨,1
...,...,...,...
14967,부대찌개_pred_node,차돌박이,1
14968,부대찌개_pred_node,찹쌀가루,1
14969,부대찌개_pred_node,청양고춧가루,1
14970,부대찌개_pred_node,파프리카추출색소,1


In [564]:
y = result.groupby([result['제품명'],result['원재료명']]).sum()['함량(%)'].unstack()
y = y.fillna(0)

In [565]:
y.reset_index(inplace=True)

In [566]:
y

원재료명,제품명,2배양조식초,5'-구아닐산이나트륨,5'-리보뉴클레오티드나트륨,5'-리보뉴클레오티드이나트륨,5'-리보뉴클레티드이나트륨,5'-이노신산이나트륨,6가지과채동치미농축액,6가지야채추출베이스,6가지야채추출베이스-1,...,훈연멸치엑기스,훈연풍미베이스,휘핑크림,흑후추,흑후추가루,흑후추분,흑후추분말,흑후추분말(조분),흑후추분태,히코리스모크오일RS30
0,(세븐일레븐PB) 돌돌브리또_비프앤치즈 데리야끼소스,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,(세븐일레븐PB) 돌돌브리또_비프앤치즈 스위트칠리소스,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,(세븐일레븐PB) 돌돌브리또_치킨앤치즈 뉴칠리타코소스,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,(세븐일레븐PB) 돌돌브리또_치킨앤치즈 프리미엄피자소스,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,(올가)불고기계란 주먹밥 불고기맛소스,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,황금밥알 스파이시포크 매운맛소스,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
866,황태밑국물소스,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
867,황태콩나물국용소스&건더기,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
868,황태해장국,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [574]:
A = y[y['제품명']=='강된장_pred_node']
B = y[y['제품명']=='부대찌개_pred_node']
C = y[y['제품명']=='순두부찌개_pred_node']
gold = pd.concat([A,B,C],axis=0)

In [597]:
gold.to_csv('lees.csv')